In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.8 MB/s eta 0:00:00


In [ ]:
from Bio import SeqIO

In [ ]:
def extract_promoter_sequences(gff_file, fasta_file, upstream_length, output_file):
    # Read in the genome assembly file and store sequences in a dictionary
    genome_dict = {}
    with open(fasta_file) as f:
        sequence = ""
        header = ""
        for line in f:
            if line.startswith(">"):
                if header != "":
                    genome_dict[header] = sequence
                header = line.strip()[1:]
                sequence = ""
            else:
                sequence += line.strip()
        genome_dict[header] = sequence
    
    # Create a dictionary to store the start and end positions of each gene
    gene_dict = {}
    with open(gff_file) as f:
        for line in f:
            if not line.startswith("#"):
                fields = line.strip().split("\t")
                if fields[2] == "gene":
                    gene_id = fields[8].split(";")[0].split("=")[1]
                    start = int(fields[3])
                    end = int(fields[4])
                    strand = fields[6]
                    if strand == "+":
                        promoter_start = max(1, start - upstream_length)
                        promoter_end = start - 1
                    elif strand == "-":
                        promoter_start = end + 1
                        promoter_end = min(len(genome_dict[fields[0]]), end + upstream_length)
                    gene_dict[gene_id] = {"start": promoter_start, "end": promoter_end, "strand": strand}
    
    # Extract promoter sequences
    with open(output_file, "w") as f:
        for gene_id in gene_dict:
            start = gene_dict[gene_id]["start"]
            end = gene_dict[gene_id]["end"]
            strand = gene_dict[gene_id]["strand"]
            if strand == "+":
                promoter_seq = genome_dict[fields[0]][start-1:end]
            elif strand == "-":
                promoter_seq = reverse_complement(genome_dict[fields[0]][start-1:end])
            f.write(f">{gene_id}\n{promoter_seq}\n")


In [ ]:
def reverse_complement(seq):
    complement = {"A": "T", "C": "G", "G": "C", "T": "A", "N": "N"}
    seq = seq.upper()
    return "".join(complement[base] for base in reversed(seq))


In [ ]:
extract_promoter_sequences("/content/gene_annotation.gff3", "/content/genome_assembly.fa", 2000, "output_file.fasta")


In [ ]:
import re

In [ ]:
def find_dreb1a_binding_sites(promoter_file, motif_file, output_file):
    # Read promoter sequences from input file
    with open(promoter_file, "r") as f:
        promoter_seqs = f.readlines()

    # Search for DREB1A binding site in each promoter sequence
    with open(output_file, "w") as f:
        for i in range(0, len(promoter_seqs), 2):
            promoter_id = promoter_seqs[i].strip()[1:]
            promoter_seq = promoter_seqs[i+1].strip()
            binding_sites = [match.start() for match in re.finditer(r"CACCGAC", promoter_seq)]

            # Write binding sites to output file
            if binding_sites:
                f.write(f">{promoter_id}\n")
                for site in binding_sites:
                    f.write(f"{site+1}\t{promoter_seq[site:site+7]}\n")


In [ ]:
find_dreb1a_binding_sites("/content/output_file.fasta", "/content/MA0971.1.meme", "downstream.fasta")


In [ ]:
def identify_dreb1a_binding_sites(promoter_file, motif_file, output_file):
    # Read promoter sequences from input file
    with open(promoter_file, "r") as f:
        promoter_seqs = f.readlines()
    with open(motif_file, "r") as f:
        motif_seq = f.readlines()[1].strip()
# Search for motif in each promoter sequence
    with open(output_file, "w") as f:
        for i in range(0, len(promoter_seqs), 2):
            promoter_id = promoter_seqs[i].strip()[1:]
            promoter_seq = promoter_seqs[i+1].strip()
            motif_sites = [match.start() for match in re.finditer(motif_file, promoter_seq)]

            # Write motif sites to output file
            if motif_sites:
                f.write(f">{promoter_id}\n")
                for site in motif_sites:
                    f.write(f"{site+1}\t{promoter_seq[site:site+len(motif_seq)]}\n")

In [ ]:
identify_dreb1a_binding_sites("/content/output_file.fasta", "/content/MA0971.1.meme", "downstream2.fasta")

In [ ]:
from Bio import motifs, SeqIO
from Bio.Alphabet import generic_dna

def search_pwm_in_sequence(seq_file, pwm_file):
    # Load the DREB1A PWM from a MEME file
    with open(pwm_file) as f:
        m = motifs.parse(f, "MEME")
        pwm = m[0].pwm

    # Load the promoter sequence
    promoter = SeqIO.read(seq_file, "fasta").seq

    # Create a Bio.Seq object with a DNA alphabet
    seq = promoter.upper()
    seq = seq.transcribe()  # convert T to U for RNA sequences
    seq = seq.back_transcribe()  # convert U back to T for DNA sequences
    seq = seq.reverse_complement()  # reverse complement the sequence if necessary
    seq = seq.upper()
    seq = seq.tomutable()
    seq.alphabet = generic_dna

    # Search for the motif in the promoter sequence
    hits = pwm.search(seq)

    # Get the sequences and positions of the hits
    hit_seqs = []
    for hit in hits:
        start = hit[0]
        end = hit[1]
        score = hit[2]
        hit_seq = seq[start:end]
        hit_seq_str = str(hit_seq)
        hit_seqs.append((start, end, hit_seq_str))

    return hit_seqs


ImportError: ignored